# CLASIFICACIÓN DE TEXTO

Se recorrerá todo el proceso desde traer el dataset hasta proceder a dicha clasificación. Durante la actividad se llevarán a cabo muchos procesos como la creación de un vocabulario, el uso de embeddings y la creación de modelos.

Las cuestiones presentes en esta actividad están basadas en un Notebook creado por François Chollet, uno de los creadores de Keras y autor del libro "Deep Learning with Python".

En este Notebook se trabaja con el dataset "Newsgroup20" que contiene aproximadamente 20000 mensajes que pertenecen a 20 categorías diferentes.

El objetivo es entender los conceptos que se trabajan y ser capaz de hacer pequeñas experimentaciones para mejorar el Notebook creado.

# Librerías

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Descarga de Datos

In [ ]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

17329808/17329808 [==============================] - 31s 2us/step


In [ ]:
import os
import pathlib

#Estructura de directorios del dataset
data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

Number of directories: 20
Directory names: ['sci.electronics', 'sci.space', 'alt.atheism', 'soc.religion.christian', 'misc.forsale', 'talk.politics.mideast', 'rec.sport.hockey', 'sci.med', 'comp.os.ms-windows.misc', 'talk.politics.guns', 'rec.motorcycles', 'comp.windows.x', 'comp.sys.mac.hardware', 'rec.sport.baseball', 'rec.autos', 'sci.crypt', 'talk.politics.misc', 'comp.graphics', 'talk.religion.misc', 'comp.sys.ibm.pc.hardware']


In [ ]:
print(data_dir)

/root/.keras/datasets/20_newsgroup


In [ ]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
fnames = os.listdir(data_dir / "comp.graphics")
total_tokens=0
for i in (range(15)):
    doc = nlp(pathlib.Path(data_dir / "comp.graphics" / fnames[i]).read_text(encoding="latin-1"))
    doc.__len__()
    total_tokens+=doc.__len__()

average_tokens = total_tokens/15

print(average_tokens)

290.73333333333335


In [ ]:
#Algunos archivos de la categoria "com.graphics"
fnames = os.listdir(data_dir / "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of files in comp.graphics: 1000
Some example filenames: ['39048', '38295', '38549', '38228', '38700']


In [ ]:
#Ejemplo de un texto de la categoría "com.graphics"
print(open(data_dir / "comp.graphics" / "37261").read())

#lines = open(data_dir / "comp.graphics" / "37261").read().split("\n")
#lines = lines[10:]
#content = "\n".join(lines)
#print(content)

Xref: cantaloupe.srv.cs.cmu.edu comp.graphics:37261 alt.graphics:519 comp.graphics.animation:2614
Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!ogicse!uwm.edu!zaphod.mps.ohio-state.edu!darwin.sura.net!dtix.dt.navy.mil!oasys!lipman
From: lipman@oasys.dt.navy.mil (Robert Lipman)
Newsgroups: comp.graphics,alt.graphics,comp.graphics.animation
Subject: CALL FOR PRESENTATIONS: Navy SciViz/VR Seminar
Message-ID: <32850@oasys.dt.navy.mil>
Date: 19 Mar 93 20:10:23 GMT
Article-I.D.: oasys.32850
Expires: 30 Apr 93 04:00:00 GMT
Reply-To: lipman@oasys.dt.navy.mil (Robert Lipman)
Followup-To: comp.graphics
Distribution: usa
Organization: Carderock Division, NSWC, Bethesda, MD
Lines: 65


			CALL FOR PRESENTATIONS
	
      NAVY SCIENTIFIC VISUALIZATION AND VIRTUAL REALITY SEMINAR

			Tuesday, June 22, 1993

	    Carderock Division, Naval Surface Warfare Center
	      (formerly the David Taylor Research Center)
			  Bethesda, Maryland

SPONSOR: NESS (Navy Engineering Software System) is sponsori

In [ ]:
#Algunos archivos de la categoria "talk.politics.misc"
fnames = os.listdir(data_dir / "talk.politics.misc")
print("Number of files in talk.politics.misc:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of files in talk.politics.misc: 1000
Some example filenames: ['176868', '176922', '178817', '178883', '179091']


In [ ]:
#Ejemplo de un texto de la categoría "talk.politics.misc"
print(open(data_dir / "talk.politics.misc" / "178463").read())

Xref: cantaloupe.srv.cs.cmu.edu talk.politics.guns:54219 talk.politics.misc:178463
Newsgroups: talk.politics.guns,talk.politics.misc
Path: cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!usenet.ins.cwru.edu!agate!spool.mu.edu!darwin.sura.net!martha.utcc.utk.edu!FRANKENSTEIN.CE.UTK.EDU!VEAL
From: VEAL@utkvm1.utk.edu (David Veal)
Subject: Re: Proof of the Viability of Gun Control
Message-ID: <VEAL.749.735192116@utkvm1.utk.edu>
Lines: 21
Sender: usenet@martha.utcc.utk.edu (USENET News System)
Organization: University of Tennessee Division of Continuing Education
References: <1qpbqd$ntl@access.digex.net> <C5otvp.ItL@magpie.linknet.com>
Date: Mon, 19 Apr 1993 04:01:56 GMT

[alt.drugs and alt.conspiracy removed from newsgroups line.]

In article <C5otvp.ItL@magpie.linknet.com> neal@magpie.linknet.com (Neal) writes:

>   Once the National Guard has been called into federal service,
>it is under the command of the present. Tha N

In [ ]:
list_all_dir = [
    'alt.atheism',
    'comp.graphics',
    'comp.sys.mac.hardware',
    'comp.windows.x',
    'misc.forsale',
    'rec.autos',
    'rec.sport.baseball',
    'rec.sport.hockey',
    'sci.crypt',
    'sci.med',
    'sci.space',
    'soc.religion.christian',
    'talk.politics.guns',
    'talk.politics.misc',
    'talk.religion.misc'
]

In [ ]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in list_all_dir:
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.misc', 'talk.religion.misc']
Number of samples: 14997


# Mezclando los datos para separarlos en Traning y Test

In [ ]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)
keras.utils.set_random_seed(seed)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples   = samples[-num_validation_samples:]
train_labels  = labels[:-num_validation_samples]
val_labels    = labels[-num_validation_samples:]

In [ ]:
print(num_validation_samples)

2999


In [ ]:
print(train_samples[:1])

['Organization: University of North Carolina, Chapel Hill\nKeywords: Frequently Asked Questions\nApproved: news-answers-request@MIT.Edu\nSupersedes: <intro_730956346@cs.unc.edu>\nNNTP-Posting-Host: mahler.cs.unc.edu\nLines: 310\n\nArchive-name: space/intro\nLast-modified: $Date: 93/04/01 14:39:10 $\n\n    FREQUENTLY ASKED QUESTIONS ON SCI.SPACE/SCI.ASTRO\n\n    INTRODUCTION\n\n    This series of linked messages is periodically posted to the Usenet\ngroups sci.space and sci.astro in an attempt to provide good answers to\nfrequently asked questions and other reference material which is worth\npreserving. If you have corrections or answers to other frequently asked\nquestions that you would like included in this posting, send email to\nleech@cs.unc.edu (Jon Leech).\n\n    If you don\'t want to see the FAQ, add \'Frequently Asked Questions\' to\nyour KILL file for this group (if you\'re not reading this with a newsreader\nthat can kill articles by subject, you\'re out of luck).\n\n    The 

In [ ]:
print(val_samples[:1])

['\nI\'ve noticed that is has become fashionable lately in rsb to predict\nthe Marlines to finish ahead of the Cubs....how?\n\nFirst Base:\n\nGrace vs Destrade...Could Destrade be the second coming of Cecil\nFielder? I doubt it. If Destrade performs to the height of expectations,\nthen even, otherwise, edge to Cubs\n\nSecond Base:\n\nSandberg vs Barberie...No contest. Sandberg will be back May 1. Edge\nto Cubs...a big edge.\n\nShortstop:\n\nVizcaino vs Weiss...Vizcaino is excellent defensively, but is an\nautomatic out at bat. Weiss isn\'t much better with the stick. Even.\n\nThird Base:\n\nDave Magadan vs Buechelle...Magadan has a higher OBP and is a better hitter.\nBuechelle has more power and is better defensively (I think) Edge to\nFlorida.\n\nCatcher: \n\nSantiago vs Wilkins...Wilkins is OK, but Santiago is better. Edge to\nMarlins\n\nLeft Field:\n\nConine vs Maldonado...Wow! 4 for 4 yesterday...I know Conine has potential.\nI watched him play at Omaha the last couple years. Until

In [ ]:
print(train_labels[:1])

[10]


In [ ]:
print(val_labels[:1])

[6]


# Tokenización de las palabras con TextVectorization

In [ ]:
from tensorflow.keras.layers import TextVectorization
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [ ]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

In [ ]:
len(vectorizer.get_vocabulary())

20000

# Viendo la salida de Vectorizer

In [ ]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([   2, 3732, 2072,   18,    2, 4523])

In [ ]:
output

<tf.Tensor: shape=(1, 200), dtype=int64, numpy=
array([[   2, 3732, 2072,   18,    2, 4523,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   

In [ ]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
test = ["the", "cat", "sat", "on", "the", "mat"]
[word_index[w] for w in test]

[2, 3732, 2072, 18, 2, 4523]

# Tokenización de los datos de entrenamiento y validación

In [ ]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

# Creación y entrenamiento del modelo. Red Neuronal Clásica

In [ ]:
modeloClasico = keras.models.Sequential()
modeloClasico.add(keras.layers.Embedding(20000, 10, input_length=200))
modeloClasico.add(keras.layers.Flatten())
modeloClasico.add(keras.layers.Dense(512, activation='relu'))
modeloClasico.add(keras.layers.Dropout(0.3))
modeloClasico.add(keras.layers.Dense(len(class_names), activation='softmax'))

In [ ]:
modeloClasico.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modeloClasico.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
modeloClasico.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))
print(modeloClasico.summary())

Epoch 1/20
94/94 [==============================] - 14s 117ms/step - loss: 2.6176 - acc: 0.1269 - val_loss: 2.4640 - val_acc: 0.1841
Epoch 2/20
94/94 [==============================] - 11s 118ms/step - loss: 2.1166 - acc: 0.3046 - val_loss: 1.8674 - val_acc: 0.3371
Epoch 3/20
94/94 [==============================] - 7s 71ms/step - loss: 1.4670 - acc: 0.5279 - val_loss: 1.4322 - val_acc: 0.4935
Epoch 4/20
94/94 [==============================] - 5s 52ms/step - loss: 0.9753 - acc: 0.7052 - val_loss: 1.1646 - val_acc: 0.5722
Epoch 5/20
94/94 [==============================] - 6s 62ms/step - loss: 0.6455 - acc: 0.8122 - val_loss: 0.9769 - val_acc: 0.6536
Epoch 6/20
94/94 [==============================] - 4s 40ms/step - loss: 0.4289 - acc: 0.8799 - val_loss: 0.9231 - val_acc: 0.6712
Epoch 7/20
94/94 [==============================] - 3s 32ms/step - loss: 0.3012 - acc: 0.9131 - val_loss: 0.8769 - val_acc: 0.6999
Epoch 8/20
94/94 [==============================] - 3s 30ms/step - loss: 0.2212

# Evaluación

In [ ]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = modeloClasico(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [['this message is about computer graphics and 3D modeling']]
)

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 167ms/step


'comp.graphics'

In [ ]:
probabilities = end_to_end_model.predict(
    [["politics and federal courts law that people understand with politician and elects congressman"]]
)

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 55ms/step


'talk.politics.guns'

In [ ]:
probabilities = end_to_end_model.predict(
    [["we are talking about religion"]]
)

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 31ms/step


'comp.windows.x'